In [13]:
from enum import Enum



[('k', 'v')]

In [3]:
import os
from typing import Literal, Any
from pathlib import Path
import sqlite3
import json
from enum import Enum

class AdvEnum(Enum):
    def __new__(cls, *args, **kwds):
        obj = object.__new__(cls)
        obj._value_ = args[0]
        return obj
    
    def __str__(self) -> str:
        default = super().__str__()
        start = default.find(self.name)
        return default[start:]

STAR: Literal['*'] = '*'
class SqlDType(Enum):
    TEXT = 'TEXT'
    INT = 'INTEGER'
    REAL = 'REAL'

LX_DIR = ".lx"
class Project:
    class Tables(AdvEnum):
        def __init__(self, val, fields) -> None:
            self.fields = [(k, d) for k, d in fields]
            self._sql = f"""\
            CREATE TABLE IF NOT EXISTS {self._table_data} (
                key TEXT,
                value TEXT,
                PRIMARY KEY key
            
            """

        DATA = "data", [("k", SqlDType.TEXT)]
        EX = "executions", [("k", SqlDType.TEXT)]

    def __init__(self, workspace: Path|str) -> None:
        workspace = Path(workspace)
        self._lx_dir = workspace.joinpath(LX_DIR)
        if not self._lx_dir.exists(): os.makedirs(self._lx_dir, exist_ok=True)
        self._con: sqlite3.Connection|None = None
        self._cur: sqlite3.Cursor|None = None

    def __enter__(self):
        self._con = sqlite3.connect(self._lx_dir.joinpath("state.db"))
        self._cur = self._con.cursor()
        for table in self.Tables:
            self._execute(f"""\
                CREATE TABLE IF NOT EXISTS {self._table_data} (
                    key TEXT,
                    value TEXT,
                    PRIMARY KEY key
                )
            """)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(exc_type, exc_val, exc_tb)
        if self._cur is not None: self._cur.close()
        if self._con is not None: self._con.close()

    def _execute(self, sql: str, values: tuple|None=None):
        assert self._cur is not None
        return self._cur.execute(sql) if values is None else self._cur.execute(sql, values)

    def _insert(self, key: str, val: dict):
        sval = json.dumps(val, separators=(",", ":"))
        self._execute(f"""\
            INSERT INTO {self._table_data_manifest} (key, value) VALUES (?, ?)
        """, (key, sval))

    def RegisterDataInstance(self, dtype: str, value: Any):
        pass
        
with Project("./cache/man_test01") as p:
    p.RegisterDataInstance("test", "x")
    for i in range(int(1e5)):
        print(f"\r{i}", end="")
        c = p._con.cursor()
    print()

9999
None None None


In [ ]:
class DataType:
    def __init__(self, attributes:Iterable[str]) -> None:
        self.Attributes: set[str] = set(attributes)

    def __eq__(self, __value: object) -> bool:
        if not isinstance(__value, type(self)): return False
        return __value.Attributes.issubset(self.Attributes)

    def __str__(self) -> str:
        return json.dumps(self.Attributes)

    def __repr__(self) -> str:
        return f"DT:{self}"